In [82]:
# Load Packages

import os
import shutil
import re

In [ ]:
# Define Functions

In [83]:
def create_voice_folder(path, name):
    if os.path.exists(os.path.join(path,name)): shutil.rmtree(os.path.join(path,name))
    os.mkdir(os.path.join(path,name))
    os.mkdir(os.path.join(path,name,'wavs'))
    os.mkdir(os.path.join(path,name,'wavs','train'))
    os.mkdir(os.path.join(path,name,'wavs','validation'))
    os.mkdir(os.path.join(path,name,'wavs','archive'))
    os.mkdir(os.path.join(path,name,'wavs','archive','train'))
    os.mkdir(os.path.join(path,name,'wavs','archive','validation'))
    os.mkdir(os.path.join(path,name,'filelists'))
    os.mkdir(os.path.join(path,name,'filelists','audio'))
    os.mkdir(os.path.join(path,name,'filelists','mel'))
    os.mkdir(os.path.join(path,name,'filelists','archive'))
    os.mkdir(os.path.join(path,name,'wavs','retakes'))
    os.mkdir(os.path.join(path,name,'wavs','retakes','train'))
    os.mkdir(os.path.join(path,name,'wavs','retakes','validation'))
    os.mkdir(os.path.join(path,name,'filelists','retakes'))
    os.mkdir(os.path.join(path,name,'filelists','retakes','train'))
    os.mkdir(os.path.join(path,name,'filelists','retakes','validation'))
    os.mkdir(os.path.join(path,name,'wavs','edited'))
    os.mkdir(os.path.join(path,name,'wavs','edited','train'))
    os.mkdir(os.path.join(path,name,'wavs','edited','validation'))

In [84]:
def enumerate_recordings(inputpath, outputpath, fileprefix, train):

    inputfiles=os.listdir(inputpath)

    removefilewithextension=['.ini','.pkf']

    for inputfilename in inputfiles:
        for remove in removefilewithextension:
            if inputfilename.endswith(remove):
                os.remove(os.path.join(inputpath, inputfilename))
                
    if train: subfolder='train'
    else: subfolder='validation'
        
    start=len(os.listdir(os.path.join(outputpath,subfolder)))+1
            
    for i, filename in enumerate(inputfiles,start=start):
                
        outputfilename=f"""{fileprefix}_{i}.wav"""
                
        shutil.copy2(os.path.join(inputpath,inputfilename), os.path.join(outputpath,subfolder,outputfilename))

In [102]:
def create_filelist(inputpath, outputpath, voicefoldername, fileprefix, audio, train, linecount):

    if audio: 
        audioormel='audio'
        ext='.wav'
    else: 
        audioormel='mel'
        ext='.pt'
        
    if train: trainorvalidation='train'
    else: trainorvalidation='validation'
                
    filelistname=str.lower(fileprefix)+'s_'+audioormel+'_text_'+trainorvalidation+'_filelist.txt'
    
    with open(inputpath,'r') as f:
        inputlines=f.readlines()
        outputlines=[]
        f.close()

    for i, inputline in enumerate(inputlines,start=1):
        if i<=linecount:
            filenumber=int(re.search('_(.*).wav', inputline).group(1))
            aftersplit=inputline.split("|",1)[1]
            outputline=f"""{voicefoldername}/{audioormel}/{trainorvalidation}/{fileprefix}_{filenumber}{ext}"""+'|'+aftersplit
            outputlines.append(outputline)
        
    fulloutputpath=os.path.join(outputpath,voicefoldername,'filelists',audioormel,filelistname)

    with open(fulloutputpath, 'w') as f:
        for outputline in outputlines:
            f.write(outputline)
        f.close()

In [86]:
def create_retakes_filelist(inputpath, inputfilelistname, outputfilelistname, train, specialstartingcharacter, retakeindices):
    
    try: os.mkdir(os.path.join(filelistpath,'retakes'))
    except: 'retakes directory already exists'

    if train: trainorvalidation='train'
    else: trainorvalidation='validation'
        
    outputpath=os.path.join(filelistpath,'retakes',trainorvalidation)
        
    try: os.mkdir(outputpath)
    except: f"""retakes/{trainorvalidation} directory already exists""" 
    
    with open(os.path.join(filelistpath, inputfilelistname),'r') as f:
        inputlines=f.readlines()
    f.close()
               
    outputlines=[]
    
    for i, inputline in enumerate(inputlines):
        if specialstartingcharacter != '':
            if inputline[0]==specialstartingcharacter: 
                outputlines.append(inputline.split(f"""{trainorvalidation}/""",1)[1])
                retakeindices.append(i+1)
        elif int(re.search('_(.*).wav', inputline).group(1)) in retakeindices:
            outputlines.append(inputline.split(f"""{trainorvalidation}/""",1)[1])
    
    with open(os.path.join(outputpath, outputfilelistname), 'w') as f:
        for outputline in outputlines:
            f.write(outputline)
        f.close()

In [87]:
def swap_retakes(audiopath, train, retakefilelistpath):
    
    with open(retakefilelistpath,'r') as f:
        retakeindices=[]
        lines=f.readlines()
        for i, line in enumerate(lines):
            retakeindices.append(int(re.search('_(.*).wav', line).group(1)))
    f.close()
        
    if train: trainorvalidation='train'
    else: trainorvalidation='validation'
        
    for i, filename in enumerate(os.listdir(os.path.join(audiopath,trainorvalidation)),start=1):
    
        if (int(re.search('_(.*).wav', filename).group(1)) in retakeindices):
            
            # Archive Old Voice Files Where Retakes Exist
                
            try: os.remove(os.path.join(outputpath,filename))
            except: print("file doesn't exists in archive table")
        
            os.rename(os.path.join(audiopath,trainorvalidation,filename), os.path.join(audiopath,'archive',trainorvalidation,filename))  
            
            # Copy Retakes
            
            shutil.copy2(os.path.join(audiopath,'retakes',trainorvalidation,filename), os.path.join(audiopath,trainorvalidation,filename))

In [99]:
# Set Environment Path

environmentpath='<insert environment path>'

In [ ]:
# Example Application

In [101]:
# Create Voice Folder

name='AC-Voice-Cloning-Data-Test'

create_voice_folder(environmentpath, name)

In [90]:
# Enumerate Recordings - Train

inputpath=f"""{environmentpath}<path to folder with recordings in chronological order>"""
outputpath=f"""{environmentpath}AC-Voice-Cloning-Data-Test\\wavs\\"""
fileprefix="AC"
train=True

enumerate_recordings(inputpath, outputpath, fileprefix, train)

In [2]:
# Check Audio File Count

targettrainfilecount=1250

filetype='train'

trainfilecount=len(os.listdir(os.path.join(outputpath,filetype)))

if trainfilecount>targettrainfilecount: print(f"""Output {filetype} file count of {trainfilecount} is greater than required count of {targettrainfilecount}; identify and remove duplicate recordings""")
elif trainfilecount<targettrainfilecount: print(f"""Output {filetype} file count of {trainfilecount} is less than required count of {targettrainfilecount}; identify and add missing recordings""")
else: print(f"""Output {filetype} file count of {trainfilecount} meets required count of {targettrainfilecount}""")

In [92]:
# Enumerate Recordings - Validation

inputpath=f"""{environmentpath}<path to folder with recordings in chronological order>"""
outputpath=f"""{environmentpath}AC-Voice-Cloning-Data-Test\\wavs\\"""
fileprefix="AC"
train=False

enumerate_recordings(inputpath, outputpath, fileprefix, train)

In [93]:
# Check Audio File Count

targettrainfilecount=100

filetype='validation'

trainfilecount=len(os.listdir(os.path.join(outputpath,filetype)))

if trainfilecount>targettrainfilecount: print(f"""Output {filetype} file count of {trainfilecount} is greater than required count of {targettrainfilecount}; identify and remove duplicate recordings""")
elif trainfilecount<targettrainfilecount: print(f"""Output {filetype} file count of {trainfilecount} is less than required count of {targettrainfilecount}; identify and add missing recordings""")
else: print(f"""Output {filetype} file count of {trainfilecount} meets required count of {targettrainfilecount}""")

Output validation file count of 100 meets required count of 100


In [94]:
# Create Audio Filelist

# Train

inputpath=f"""{environmentpath}AC-Voice-Cloning-Data\\filelists\\archive\\acs_audio_text_train_filelist_1250.txt"""
voicefoldername='AC-Voice-Cloning-Data-Test'
fileprefix='AC'
audio=True
train=True
linecount=1250

create_filelist(inputpath, environmentpath, voicefoldername, fileprefix, audio, train, linecount)

# Validation

inputpath=f"""{environmentpath}AC-Voice-Cloning-Data\\filelists\\archive\\acs_audio_text_validation_filelist_100.txt"""
train=False

create_filelist(inputpath, environmentpath, voicefoldername, fileprefix, audio, train, linecount)

In [97]:
# Optional - Build Retakes Filelist

filelistpath=f"""{environmentpath}AC-Voice-Cloning-Data-Test\\filelists"""
inputfilelistname='retakes\\train\\acs_audio_text_train_filelist.txt'
outputfilelistname='acs_audio_retakes_train_filelist.txt'
train=True
specialstartingcharacter='#'
retakeindices=[]

create_retakes_filelist(filelistpath, inputfilelistname, outputfilelistname, train, specialstartingcharacter, retakeindices)

In [1]:
# Optional - Swap Retake Audio Files

audiopath=f"""{environmentpath}AC-Voice-Cloning-Data-Test\\wavs"""
train=True
retakefilelistpath=f"""{environmentpath}AC-Voice-Cloning-Data-Test\\filelists\\retakes\\train\\acs_audio_retakes_train_filelist.txt"""

swap_retakes(audiopath, train, retakefilelistpath)